<h3>VVI</h3>

In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain.agents import tool
from langchain.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-4o-mini")

In [4]:
@tool
def getWordLength(word: str) -> int:
    """
        Returns the Length of a Word
    """
    return len(word)

In [5]:
getWordLength.invoke(input={"word":"iku"})

3

In [6]:
tools=[getWordLength]

In [8]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a very powerful assistant, but don't know current events"),
        ("user","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [13]:
llmWithTools=llmOpenAI.bind_tools(tools=tools)

In [14]:
agent={
        "input":lambda x: x['input'],
        "agent_scratchpad":lambda x: format_to_openai_tool_messages(x['intermediate_steps']) 
        }\
        |prompt\
        |llmWithTools\
        |OpenAIToolsAgentOutputParser()

In [15]:
agentExecutor=AgentExecutor(agent=agent, tools=tools,verbose=True)

In [16]:
agentExecutor.invoke(input={"input":"How many letters are there in `Eklavya`"})



> Entering new AgentExecutor chain...

Invoking: `getWordLength` with `{'word': 'Eklavya'}`


7The word "Eklavya" has 7 letters.

> Finished chain.


{'input': 'How many letters are there in `Eklavya`',
 'output': 'The word "Eklavya" has 7 letters.'}

<h3> Adding Memories </h3>

In [17]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a very powerful assistant, but don't know current events"),
        ("placeholder","{chat_history}"),
        ("user","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [18]:
agent={
        "input":lambda x: x['input'],
        "chat_history": lambda x: x['chat_history'],
        "agent_scratchpad":lambda x: format_to_openai_tool_messages(x['intermediate_steps']) 
        }\
        |prompt\
        |llmWithTools\
        |OpenAIToolsAgentOutputParser()

In [19]:
agentExecutor=AgentExecutor(agent=agent, tools=tools,verbose=True)

In [20]:
messageHistory=ChatMessageHistory()

In [23]:
agentExecutorWithHistory=RunnableWithMessageHistory(
    get_session_history=lambda session_id:messageHistory,
    runnable=agentExecutor,
    input_messages_key="input",
    history_messages_key="chat_history",                   
)

In [21]:
session_id="<iku>"

In [24]:
agentExecutorWithHistory.invoke(
                                input={'input':"What is the total length for the word Tania?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...

Invoking: `getWordLength` with `{'word': 'Tania'}`


5The total length of the word "Tania" is 5 letters.

> Finished chain.


{'input': 'What is the total length for the word Tania?',
 'chat_history': [],
 'output': 'The total length of the word "Tania" is 5 letters.'}

In [25]:
agentExecutorWithHistory.invoke(
                                input={'input':"What question did I ask?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
You asked, "What is the total length for the word Tania?"

> Finished chain.


{'input': 'What question did I ask?',
 'chat_history': [HumanMessage(content='What is the total length for the word Tania?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The total length of the word "Tania" is 5 letters.', additional_kwargs={}, response_metadata={})],
 'output': 'You asked, "What is the total length for the word Tania?"'}

In [26]:
agentExecutorWithHistory.invoke(
                                input={'input':"Is  this an Indian Name?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
Yes, "Tania" is a name that is used in India, as well as in various other cultures around the world. It can be a variant of the name "Tanya," which has Slavic origins but is also found in India. Names can have different meanings and significance based on cultural context.

> Finished chain.


{'input': 'Is  this an Indian Name?',
 'chat_history': [HumanMessage(content='What is the total length for the word Tania?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The total length of the word "Tania" is 5 letters.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What question did I ask?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You asked, "What is the total length for the word Tania?"', additional_kwargs={}, response_metadata={})],
 'output': 'Yes, "Tania" is a name that is used in India, as well as in various other cultures around the world. It can be a variant of the name "Tanya," which has Slavic origins but is also found in India. Names can have different meanings and significance based on cultural context.'}

In [27]:
agentExecutorWithHistory.invoke(
                                input={'input':"Is this an English Word?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
"Tania" is primarily recognized as a proper noun (a name) rather than a common English word. While it may be used in English-speaking contexts as a name, it does not have a standard definition or meaning in the English language like common nouns do.

> Finished chain.


{'input': 'Is this an English Word?',
 'chat_history': [HumanMessage(content='What is the total length for the word Tania?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The total length of the word "Tania" is 5 letters.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What question did I ask?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You asked, "What is the total length for the word Tania?"', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Is  this an Indian Name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes, "Tania" is a name that is used in India, as well as in various other cultures around the world. It can be a variant of the name "Tanya," which has Slavic origins but is also found in India. Names can have different meanings and significance based on cultural context.', additional_kwargs={}, response_metadata={})],
 'output': '"Tania" is primarily recognized as a pr